# Work with test data


In [61]:
import tensorflow.keras.models as md
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [62]:
model = md.load_model("./models/titanic.keras")

In [63]:
df = pd.read_csv("./data/test.csv")

In [64]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [65]:
features = ["Pclass", "Sex", "Age", "Fare"]
numeric = ["Pclass", "Age", "Fare"]
categoric = ["Sex"]

x = df[features]

In [66]:
scaler = MinMaxScaler().set_output(transform="pandas")
onehot = OneHotEncoder(sparse_output = False).set_output(transform="pandas")

In [67]:
x.loc[:, numeric] = scaler.fit_transform(x[numeric])

In [68]:
x = pd.concat([x, onehot.fit_transform(x[categoric])], axis=1).drop(columns = ["Sex"])

In [69]:
x = pd.DataFrame(SimpleImputer(strategy = "median").fit_transform(x), columns = x.columns)

In [70]:
x.head()

,Pclass,Age,Fare,Sex_female,Sex_male
0,1.0,0.452723,0.015282,0.0,1.0
1,1.0,0.617566,0.013663,1.0,0.0
2,0.5,0.815377,0.018909,0.0,1.0
3,1.0,0.353818,0.016908,0.0,1.0
4,1.0,0.287881,0.023984,1.0,0.0


In [71]:
predictions = np.where(model.predict(x) < 0.5, 0, 1)

14/14 [==============================] - 0s 2ms/step


In [72]:
len(df)

418

In [75]:
pd.options.display.max_rows = 999

In [76]:
df["Survived"] = predictions

In [77]:
ans = df.loc[:, ["PassengerId", "Survived"]]

In [78]:
ans.to_csv("./data/prediction.csv", index=False)